To create the table in Riak for the HVAC data.  After experimentation, a quantum of 30 days was chosen.  Dummy key fields of 'Family' and 'Series' were added to allow filtering on buildingid and systemid as secondary query fields.  This should be changed in TS 1.3

In [1]:
from riak import RiakClient, RiakNode

/home/stephen/ESL/local/lib/python2.7/site-packages/riak/security.py:45: UserWarning: OpenSSL 1.0.1f 6 Jan 2014 (>= 1.0.1g required), TLS 1.2 support: False
  warnings.warn(msg, UserWarning)


In [2]:
myClient = RiakClient(pb_port=8087, protocol='pbc')

In [3]:
myClient.ping() #test connection open

True

In [5]:
t=MyClient.table("")

NameError: name 'MyClient' is not defined

In [6]:
#Create new table with the dummy fields, does order matter? - yes, must have family, series, readingtime IN ORDER
tbname="hvacnew3"
fmt="""    
    CREATE TABLE hvacnew3 (
    family varchar not null,
    series varchar not null,
    readingtime timestamp not null,
    buildingid sint64 not null,
    systemid sint64 not null,
    systemage sint64 not null,
    targettemp sint64 not null,
    actualtemp sint64 not null,
    primary key ((family,series,quantum(readingtime,30,'d')),family,series,readingtime))
"""

In [7]:
myClient.ts_query(tbname,fmt)

In [8]:
myClient.ts_describe('hvacnew3') #gives a ts_object to get at details need to access the rows property

In [9]:
for r in myClient.ts_describe('hvacnew3').rows:
    print r

['family', 'varchar', False, 1L, 1L]
['series', 'varchar', False, 2L, 2L]
['readingtime', 'timestamp', False, 3L, 3L]
['buildingid', 'sint64', False, None, None]
['systemid', 'sint64', False, None, None]
['systemage', 'sint64', False, None, None]
['targettemp', 'sint64', False, None, None]
['actualtemp', 'sint64', False, None, None]


In [10]:
table=myClient.table('hvacnew3') #set the table

In [11]:
print table

hvacnew3


In [13]:
import csv #source data is CSV so need lib to access it efficiently

In [14]:
#Creation of rows and storing in one operation.  Per documentation should be OK for maybe 10k rows
drows=[]
with open('HVAC-ms.csv','rU') as csvfile:
    dreader=csv.reader(csvfile)
    for r in dreader:
        drows.append([str(r[6]),str(r[7]),int(r[0]),int(r[5]),int(r[3]),int(r[4]),int(r[1]),int(r[2])])
    #for r2 in drows:
    #    print r2
    tso=table.new(drows)
    print "Store result: ",tso.store()        

Store result:  True
